# 最初に

### 本エントリーについて
・本エントリーは、「YOLOの論文紹介」になります。そのため、「実際にやってみた」といった内容を含みません。  
・本エントリー執筆時点で、YOLOはv3まで出ていますが、その原点となる最初の「YOLO」についての紹介です。

### YOLOについて
画像認識のアルゴリズムで、2016年に発表された。  
リアルタイムで処理かつ高精度で処理できる。  
また、１つのCNNネットワークで処理を完結しているため、学習が比較的容易。  
コードの商用利用不可。  
[論文はこちら](https://arxiv.org/pdf/1506.02640.pdf)

# 目次
1.introduction  
2.Unified Detection  
3.Comparison to Other Detection Systems  
4.Experiments  
5.Real-Time Detection In The Wild  
6.Conclusion


# 1.introduction(序論)
既存の画像認識のアルゴリズムである「DPM」や「R-CNN系」は、画像の領域推定と分類が分断されており、それゆえ処理が複雑であり、かつ処理時間も長くなりがちであった。
「YOLO」では、画像認識を回帰問題に落とし込むことで「画像の領域推定」と「分類」を同時に行うことを実現した。
「YOLO」のアルゴリズムは１つのCNNで完結するためシンプルであり、また既存の手法と比較して下記のようなメリットを得ることができる。

##### メリット1.処理が早い
シンプルな回帰問題に落とし込んだことで、「複雑なパイプラインを考慮する必要がなくなった」ため処理が早い。
「通常のYOLO」（Titan X GPUを利用）の検証では45f/sec、「より早いバージョン」での検証では150f/secを発揮した。
これはリアルタイムのストリーミングビデオを25msec程度のレイテンシーで処理できるパフォーマンスである。
また、既存のリアルタイム処理アルゴリズムと比較して倍以上の精度（mAP）を発揮した

##### メリット2.画像全体を見て予測することができる
既存のアルゴリズムでは「sliding window」や「」等の手法を使って、物体の候補領域を検出しようとしていた。  
この手法のせいか、例えば「Fast R-CNN」は背景を物体だと誤検出することが多かった。  
「YOLO」では、画像全体の情報から学習や検証を実施することができるので、上記のような誤検出が「Fast R-CNN」の半分以下となっている。

##### メリット3.
YOLOは汎化性能が高い


最先端なアルゴリズムと比較すると、YOLOはまだ精度の面で劣っている。
この後の検証で確認するが、YOLOは処理速度は早い一方で「小さいもの」の検出が弱点なのである。

# 2.Unified Detection

##### 概要
YOLOでは、画像をS×Sのgridに分割する。  
各「grid　cell」はB個の「bounding boxes(物体の候補領域)」と、各「bounding boxの信頼度」を計算する。  
各「bounding box」は下記の5つのパラメータからなる。  

x:「bounding box」の中心の座標（x軸）  
y:「bounding box」の中心の座標（y軸）  
w:「bounding box」の横幅  
h:「bounding box」の高さ  
信頼度:上述の通り  


また、各「grid cell」は下記のような条件付き確率をもつ。

```math
p(その物体が何であるか | そのgrid cellに物体がある確率)
```

今回は、「PASCAL VOC」のデータセットを利用して検証を進めた。
また、本検証でのパラメータは下記の通り。

S:7
B:2
物体のクラス数:20(データセットが２０種類の物体からなるデータセットなので)


##### ネットワーク構造
序論で述べたとおり、「YOLO」は単一のCNNネットワーク構造からなる。
下記の画像を参照いただくとイメージがつきやすいかと思うが、最初のconv層(24層)で特徴量を抽出し、最後の全結合層(2層)で「分類」や「物体領域の座標修正」を行う。
# ここに画像を入れる

また、本論文では上記のとは別に「Fast YOLO」についても検証を進めた。
こちらは畳み込み層が9層になり、フィルターも少なくした以外はYOLOと同じもの。計算量が減るため処理が早い。


##### 学習
まず最初の20層の畳み込み層で事前学習をした。
この時点で、「ImageNet1000-class competition dataset」の「top-5 accuracy」で88%の結果だった。
この事前学習の後に、4層の畳み込み層と２層の全結合層を追加する形で本モデルを生成した。（初期パラメータは、乱数で適当に設定）
（このような、事前学習したモデルに後から畳み込み層と結合層を追加するやり方はパフォーマンスを上げてくれる、らしい。）
[詳細はこちら](https://arxiv.org/pdf/1504.06066.pdf)


##### 正規化
「bounding box」の座標、横幅、高さについて０から１の範囲に収まるように正規化をした。

##### optimizer

最後の結合層では線形の活性化関数を利用した。
それ以外の層では下記の活性化関数を利用した。

```math
φ(x) = x, if x > 0
            0.1x, otherwise
```

##### cost関数
簡単で使いやすいから二乗和誤差を使ったが、これは「理想ではない」と論文中では言っている。
（「分類誤差」と「候補領域抽出誤差」を同等に扱ってしまう、ほとんどのcellが物体を含まないために勾配消失が起きてしまい学習をやたらと早く収束させてしまう）

この問題に対応するために、下記の２つのパラメータを導入した。
・物体がない場合はあまり学習に反映させない
・物体があった場合の分類ミスは大きく反映させる

また、二乗和誤差だとバウンディングボックス（BB）が大きくても小さくても誤差は同じものとして計算してしまう。
本アルゴリズムとしては、大きなBBには小さなBBよりも小さな誤差としてほしいので、BBのw,hはそのままの値ではなく平方根を利用した。


##### トレーニング中に最適化した関数について

長すぎるのでここでは数式は書かないが、概要だけ記述する。  
・「分類誤差のペナルティ」は「grid cellに物体が存在する場合」のみ設ける  
・「BBの座標誤差のペナルティ」は「各grid cellでもっともIoUが高い場合」のみ設ける

下記の関数が小さくなるように学習を進めた。  

```math
(最適化する関数) =
                            (BBに物体があった場合の、x,y座標の誤差) + 
                            (BBに物体があった場合の、横幅と高さの誤差) + 
                            (BBに物体があった場合の、BBの信頼度誤差) + 
                            (BBに物体が無かった場合の、BBの信頼度誤差) + 
                            (BBに物体があった場合の、分類誤差) 
```

##### トレーニング中のあれこれ
学習係数
最初は小さく設定する。そうでないと勾配が安定しない。
最初の７５エポックは10^-2,次の３０は10^-3,最後の30は10^-4とした。

エポック数:135  
バッチサイズ:64  
モーメンタム:0.9  
decay:0.0005  

過学習の回避のために、「dropout」と「データ拡張」を行った。  
「dropout」は最初の結合層のところで、半分のノードをドロップアウトする。  
「データ拡張」は元データの２０%くらいを新しく生成した。


##### non-maximal suppression
YOLOでは、１つの物体が複数のcellから抽出されてしまうことがある。  
（大きな物体やちょうどcellの分割点に存在する物体等）  
そのような物体を「１つの物体」として認識するために「non-maximal suppression」を利用した。  


##### YOLOの制限
１つのgridで分類できる物質はパラメータとして設定した「BB数」に制限されてしまう。  
また、各BBにおける予測クラスは１つのみになってしまう。
そのため「小さな物体の集まり」（ex.鳥の大群）の識別が苦手。

また、「既存とは異なるアスペクト比を持つ物体」の検出も苦手。

<img src="./work/YOLO/network.png">

# 3.既存手法との比較

### 3-1.DPM(Deformable parts models)
YOLOでは、１つのネットワークで完結する点が異なる。  
１つのネットワークで全てを完結させることで、最終的なパフォーマンス、精度の向上に繋がる。

### 3-2.R-CNN
R-CNNやその派生系も、特徴量抽出やBB検出、物体分類等で異なるモジュールを使うのでチューニングが大変かつ、処理が遅い。  
また、YOLOはその制約により、一つのgridに識別するBBの数を制限しているので、「同じ物体を重複して出力する」というR-CNNのデメリットを和らげている。  

「Fast R-CNN」、「Faster R-CNN」でも、「リアルタイム処理」を謳うためには処理が遅い。


### 3-3.Deep MultiBox
「ある特定の物質を検知する」目的には利用できるかもしれないが、あくまでも物体検出をおこなう上でのパーツの１つに過ぎない。  
YOLOは物体検出において「より汎化的な」アルゴリズムであるとのこと。  
BBを推測する際にCNNで特徴量を抽出するところは共通点ではある。

### 3-4.OverFeat
物体検出と分類を同時にできるものではない。

### 3-5.MultiGrasp
思想は似ている。ただ、MultiGraspは処理できる内容が単純すぎる。  
（物体のサイズや他クラス分類ができない等）  
その点、YOLOは１つの画像で複数の物体＆複数のクラスへの分類が可能である。


# 4.検証
YOLOとR-CNNの派生系の中で一般的な「Fast R-CNN」の処理結果を比較する。  

### 4-1.リアルタイムで処理できるシステムとの比較  
「リアルタイムで」処理できると思われる手法は[Sadeghi et al」が提案した手法](M. A. Sadeghi and D. Forsyth. 30hz object detection with dpm v5. In Computer Vision–ECCV 2014, pages 65–79.Springer, 2014. 5, 6)で、この手法だと30fpsというパフォーマンスが証明されている。  
なので、YOLOと上記手法を比較した。他の手法は処理速度と精度の観点から比較するレベルに至っていないためである。  
「Fast YOLO」が一番処理が早いものの、精度(mAP)が52.7%であった。（既存のリアルタイム検出手法の２倍以上の精度）  
一方、YOLOはリアルタイムで処理できるパフォーマンスを維持しつつもmAPを63.4％にまで押し上げた。  
VGG-16を使ったYOLOも試したが、精度は向上したもののリアルタイム処理ができるほどのパフォーマンスが得られなかった。  
「Fastest DPM」は、処理速度がまだリアルタイム処理できるほどのものではないうえに精度がそこまで高くない。  
「R-CNN Minus R」はリアルタイム処理するとは程遠い。
「Fast R-CNN」はmAPは高いが、リアルタイム処理とは程遠い。  
「Faster R-CNN」も同様に、まだリアルタイム処理とは程遠い。


##### リアルタイム処理ができると思われる手法

| 手法 | トレーニングデータ | mAP | FPS |
|---|---|---|
|100Hz DPM | 2007 | 16.0 | 100 |
|30Hz DPM | 2007 | 26.1 | 30 |
|Fast YOLO | 2007+2012 | 52.7 | 155 |
|YOLO | 2007+2012 | 63.4 | 45 |


##### リアルタイム処理はできなさそうな手法
| 手法 | トレーニングデータ | mAP | FPS |
|---|---|---|
|Fastest DPM | 2007 | 30.4 | 15 |
|R-CNN Minus R | 2007 | 53.5 | 6 |
|Fast R-CNN | 2007+2012 | 70.0 | 0.5 |
|Faster R-CNN VGG-16 | 2007+2012 | 73.2 | 7 |
|Faster R-CNN ZF | 2007+2012 | 62.1 | 18 |
|YOLO VGG-16 | 2007+2012 | 66.4 | 21 |


### 4-2.VOC 2007 Error Analysis
最先端の手法（Fast R-CNN）とYOLOの違いをより明確にするために、「誤差の内容」を調査しました。
評価指標は[ Hoiem et al.](D. Hoiem, Y. Chodpathumwan, and Q. Dai. Diagnosing error in object detectors. In Computer Vision–ECCV 2012, pages
340–353. Springer, 2012. 6)を参考にした。  

「YOLO」は「Fast R-CNN」と比較すると「背景を物体と誤認識しない」点について高いパフォーマンスを発揮していた。  



### 4-3.Fast R-CNNとYOLOを組み合わせてみた  

YOLOは「背景を物体と認識しない」点について精度が高いので、そのメリットを活かす形で「Fast R-CNN」と組み合わせたところ、「Fast R-CNN」の精度（mAP）がちょっとだけ向上した。  
具体的には、「Fast R-CNN」が出力したBBをYOLOもBBとして出力しているか、というチェックを加えるイメージで組み合わせた。  

結局２つのモデルを処理させることになるので処理時間に問題があるのだが、YOLOの処理時間が短いので大きな問題にはならない。  
既に、「Fast R-CNN」を使っている場合には、この組み合わせを利用することで精度向上を図ることができる。  


### 4-4.VOC 2012 Results
VOC 2012のデータセットでも検証した。  
YOLOのmAPは57.9%で、他の最先端の手法よりも低かった。  
クラスごとに分類結果を見てみると、「ボトル」や「羊」等の小さい物質の検出にミスが多いようだった。  
逆に「猫」や「電車」等のクラスの分類はよくできていた。  
「Fast R-CNN」と「YOLO」を組み合わせたモデルは、トップクラスのパフォーマンスを発揮した。  


### 4-5.美術作品中の人物分類
アカデミックな世界では学習用と検証用データが「同じところから」派生するが、実社会においてはそうとはならず、学習に利用したものとは全く異なる分類をしなくてはならないため実用が難しい。  
そこで、汎用性を確認する一環として「ピカソや人物画から人間を適切に抽出できるか」、という検証を実施した。
結果は下記の通りだった。

| 手法 | VOC 2007　AP | Picasso AP| Picasso　Best F1 | People-Art　AP |
|---|---|---|---|
|YOLO | 59.2 | 53.3 | 0.590 | 45 |
|R-CNN | 54.2 | 10.4 | 0.226 | 26 |
|DPM | 43.2 | 37.8 | 0.458 | 32 |
|Poselets | 36.5 | 17.8 | 0.271 | 　 |
|D&T |   | 1.9 | 0.051 |   |


以上から、下記の通り整理できる。
・R-CNN
VOC2007については高いAPを維持できるものの、美術作品に対しては精度が低い。
「R-CNN」は「selective search」を採用しているが、このアルゴリズムは通常の画像に対して利用するのがいいであろう。

・DPM
「DPM」は美術作品に対しても精度をある程度維持できている。
ただ、元々の精度が良くない。

・YOLO
「VOC 2007」、美術作品のいずれにおいてももっとも高い精度を発揮した。
これは「YOLO」では物体のサイズ、形、周囲との関係性までモデリングできているからであろう。
美術作品と通常の画像では「ピクセルレベル」で見ると全く異なるが、「物体のサイズや形」といった面では似ているので「YOLO」はこのようなパフォーマンスを発揮できたのだろう。


# 5.Real-Time Detection In The Wild
YOLOをWebカメラに接続して、リアルタイムでのパフォーマンスを実世界で検証してみた。
結果は[こちら](http://pjreddie.com/yolo/.)をご参照ください。


# 6.Conclusion
「YOLO」は単一のネットワークで完結している点が従来の手法と異なる。
「Fast YOLO」は汎用的な物体検出としてはもっとも早いアルゴリズムであり、「YOLO」はリアルタイム画像認識の最先端の手法と言えるだろう。（2016年５月時点）
また、「YOLO」は新しいドメインの画像にも対応できるのでアプリケーションへの適用に適しているだろう。



# 備考

##### IoU(Intersection over Union)
[このブログ](https://meideru.com/archives/3538)の解説がとてもわかりやすくて良い。  


##### Non-Maximum Suppression

論文中でもさらっと出てきているが、これはなんなのか。  
[このブログ](https://meideru.com/archives/3538)の解説がとてもわかりやすくて良い。  
同じクラスと予測したBBが重複している際、一つの領域として調整するアルゴリズムのようです。

IoU値が閾値よりも大きい場合は、その領域を抑制（suppression）する、というもの。




